In [1]:
#!pip install implicit

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [3]:
import random

In [4]:
data = pd.read_csv('retail_train.csv')
data.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


In [5]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [6]:
data['week_no'] < data['week_no'].max() - test_size_weeks

0           True
1           True
2           True
3           True
4           True
           ...  
2396799    False
2396800    False
2396801    False
2396802    False
2396803    False
Name: week_no, Length: 2396804, dtype: bool

### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [7]:
def random_recommendation(items, n=5):
    """Случайные рекоммендации"""
    
    items = np.array(items)
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [8]:
def weighted_random_recommendation(data, n=5): #  возвращает массив длиной n рекомендованных взвешенных случайных товаров
    recs = [] # массив рекомендованных товаров
    r = random.sample(range(1, int(data['sales_value'].sum())), n) # массив из n случайных чисел в диапазоне от 1 до суммы весов всех товаров 
    r = np.array(r)  
    r.sort() # сортируем по возрастанию
    for item_id, weight in zip(data['item_id'], data['sales_value']): # идем по всем уникальным товарам и их весам
        r = r - weight     # на каждом шаге вычитаем из каждого числа в массиве r вес текущего товара.   
        if r.size == 0:    # если в r больше нет чисел, значит рекомендация готова
            break 
        if r[0] <= 0:            # если самое маленькое число в r стало <=0, то
            recs.append(item_id) # кладем в список рекомендованных текущий товар 
            r = np.delete(r, 0)  # удаляем это число из r
    
    return recs # тратит по 2 секунды на одну рекомендацию, что безумно долго. Просто в изначальном датасете очень много данных, по которым надо пройтись. Но по идее можно распоточить. 

In [9]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [10]:
%%time

items = data_train.item_id.unique()

result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=500))
result.head(2)

Wall time: 4.3 s


,user_id,actual,random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[45364, 1120929, 1138353, 703181, 5129574, 111..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8159629, 137605, 1027829, 7110473, 13007193, ..."


In [11]:
%%time
res = result.loc[:1] 
res['weighted_random_recommendation'] = res['user_id'].apply(lambda x: weighted_random_recommendation(data_train, n=500))
res # работает очень медленно, для всего датасета это не подойдет) Но я вроде одним циклом иду, без вложений.

Wall time: 6.47 s


<timed exec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,user_id,actual,random_recommendation,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[45364, 1120929, 1138353, 703181, 5129574, 111...","[1001521, 1135044, 1063639, 1112242, 887474, 8..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[8159629, 137605, 1027829, 7110473, 13007193, ...","[1034189, 1029743, 8090509, 819453, 6534077, 1..."


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [12]:
import re

In [13]:
result = pd.read_csv('predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[5586238, 1015228, 866118, 2416733, 2603573]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 1127831, 1098066, 826249, 878996]","[981760, 1127831, 1098066, 878996, 826249]","[981760, 1127831, 1098066, 826249, 878996]","[999999, 1082185, 1029743, 995785, 1004906]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[161354, 63027, 1027802, 12263694, 307395]","[6534178, 6533889, 1029743, 6534166, 1082185]","[981760, 995242, 1029743, 840361, 961554]","[981760, 1004906, 961554, 1096036, 1080414]","[981760, 1004906, 859075, 1096036, 961554]","[999999, 1082185, 1098066, 6534178, 1127831]"


In [14]:
len(result)

2042

In [15]:
# your_code
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    return flags.sum() / len(recommended_list)

In [16]:
def precision_at_k(recommended_list, bought_list, k=5):
    return precision(recommended_list[:k], bought_list)

In [17]:
def spr(str): # парсим строки
    return re.findall('[0-9]+', str)

In [18]:
def prec_mean(column): # считаем средний precision для отдельных столбцов. 
    prec = []
    for i in range(len(result)):
        prec.append(precision_at_k(spr(result[column][i]), spr(result['actual'][i]), k=5))
    return np.mean(prec)

In [19]:
def final(): # вывод информации в компактном виде.
    for i in result.drop(columns=['user_id']):
        print(f'mean precision_at_k for {i}: {prec_mean(i)}')

In [20]:
%%time
final()

mean precision_at_k for actual: 1.0
mean precision_at_k for random_recommendation: 0.0005876591576885408
mean precision_at_k for popular_recommendation: 0.15523996082272282
mean precision_at_k for itemitem: 0.03359451518119491
mean precision_at_k for cosine: 0.03525954946131244
mean precision_at_k for tfidf: 0.036141038197845254
mean precision_at_k for own_purchases: 0.17998694090760692
Wall time: 1.71 s


-----------------------на данных из первого пункта дз-----------------------------------

In [21]:
precision_at_k(res['random_recommendation'][0], res['actual'][0], k=5)

0.0

In [22]:
precision_at_k(res['random_recommendation'][0], res['actual'][0], k=100)

0.0

In [23]:
precision_at_k(res['weighted_random_recommendation'][0], res['actual'][0], k=5)

0.0

In [24]:
precision_at_k(res['weighted_random_recommendation'][0], res['actual'][0], k=100)

0.01

Лучше всего себя показали рекомендации из более раниих покупок пользователя, а также популярные товары. Взвешенные случайные рекомендации показывают лишь чуть лучший реультат по сравнению с чистым рандомом. Но в целом ни один подход не показал значительных успехов. 0.17 в лучшем случае. 

### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

я не понял задание. Если можно, давайте его разберем на вебинаре. 